In [1]:
from vllm import LLM, SamplingParams

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

2025-02-17 22:57:54.450495: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-17 22:57:54.457287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739833074.465432 1007209 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739833074.467707 1007209 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 22:57:54.477396: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

INFO 02-17 22:57:56 __init__.py:190] Automatically detected platform cuda.


In [2]:
# llm = LLM(model="facebook/opt-125m")
llm = LLM(model="Qwen/Qwen2.5-1.5B-Instruct")

INFO 02-17 22:58:03 config.py:542] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 02-17 22:58:03 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-1.5B-Instruct, num

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-17 22:58:06 model_runner.py:1115] Loading model weights took 2.8875 GB
INFO 02-17 22:58:12 worker.py:267] Memory profiling takes 5.47 seconds
INFO 02-17 22:58:12 worker.py:267] the current vLLM instance can use total_gpu_memory (15.99GiB) x gpu_memory_utilization (0.90) = 14.39GiB
INFO 02-17 22:58:12 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 9.43GiB.
INFO 02-17 22:58:12 executor_base.py:110] # CUDA blocks: 22071, # CPU blocks: 9362
INFO 02-17 22:58:12 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 10.78x
INFO 02-17 22:58:13 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.92it/s]

INFO 02-17 22:58:31 model_runner.py:1562] Graph capturing finished in 18 secs, took 0.00 GiB
INFO 02-17 22:58:31 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 24.55 seconds


In [3]:
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 11.58it/s, est. speed input: 63.73 toks/s, output: 185.38 toks/s]

Prompt: 'Hello, my name is', Generated text: " Kailin. I'm a 17 year old girl from Washington."
Prompt: 'The president of the United States is', Generated text: ' running for re-election. The current incumbent has a 50% chance of'
Prompt: 'The capital of France is', Generated text: ' Paris. Which of the following options correctly represents the name of the capital?\nA'
Prompt: 'The future of AI is', Generated text: ' exciting, but there are some serious challenges that need to be overcome. One of'


In [2]:
from PIL import Image

image_1_path = 'processed_output2/processed_frame_025550.jpg'
image_1 = Image.open(image_1_path)

image_2_path = 'processed_output2/processed_frame_040925.jpg'
image_2 = Image.open(image_2_path)

In [3]:
from vllm import LLM


llm = LLM(model="llava-hf/llava-1.5-7b-hf")

# Refer to the HuggingFace repo for the correct format to use
prompt = "USER: <image>\nWhat is the content of this image?\nASSISTANT:"

# Load the image using PIL.Image
image = image_1

# Single prompt inference
outputs = llm.generate({
    "prompt": prompt,
    "multi_modal_data": {"image": image},
})

for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)

# Batch inference
outputs = llm.generate(
    [
        {
            "prompt": "USER: <image>\nWhat is the content of this image?\nASSISTANT:",
            "multi_modal_data": {"image": image_1},
        },
        {
            "prompt": "USER: <image>\nWhat's the color of this image?\nASSISTANT:",
            "multi_modal_data": {"image": image_2},
        }
    ]
)

for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)

2025-02-18 09:35:44.593407: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 09:35:44.775211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739871344.849766 1034588 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739871344.871785 1034588 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 09:35:45.043036: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

INFO 02-18 09:35:47 __init__.py:190] Automatically detected platform cuda.
INFO 02-18 09:35:58 config.py:542] This model supports multiple tasks: {'reward', 'score', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.
INFO 02-18 09:35:58 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='llava-hf/llava-1.5-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-1.5-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_t

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 02-18 09:36:13 model_runner.py:1115] Loading model weights took 13.1342 GB


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


INFO 02-18 09:36:16 worker.py:267] Memory profiling takes 3.08 seconds
INFO 02-18 09:36:16 worker.py:267] the current vLLM instance can use total_gpu_memory (15.99GiB) x gpu_memory_utilization (0.90) = 14.39GiB
INFO 02-18 09:36:16 worker.py:267] model weights take 13.13GiB; non_torch_memory takes 1.51GiB; PyTorch activation peak memory takes 0.49GiB; the rest of the memory reserved for KV Cache is -0.74GiB.
INFO 02-18 09:36:16 executor_base.py:110] # CUDA blocks: 0, # CPU blocks: 512
INFO 02-18 09:36:16 executor_base.py:115] Maximum concurrency for 4096 tokens per request: 0.00x


ValueError: No available memory for the cache blocks. Try increasing `gpu_memory_utilization` when initializing the engine.